# BUSINESS PROBLEM UNDERSTANDING

Build a model that takes user's information as input and recommends 3 simillar customers based on their profile and transaction history.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

In [2]:
# Load the data
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

In [3]:
customers 

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15
...,...,...,...,...
195,C0196,Laura Watts,Europe,2022-06-07
196,C0197,Christina Harvey,Europe,2023-03-21
197,C0198,Rebecca Ray,Europe,2022-02-27
198,C0199,Andrea Jenkins,Europe,2022-12-03


In [4]:
products

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31
...,...,...,...,...
95,P096,SoundWave Headphones,Electronics,307.47
96,P097,BookWorld Cookbook,Books,319.34
97,P098,SoundWave Laptop,Electronics,299.93
98,P099,SoundWave Mystery Book,Books,354.29


In [5]:
transactions

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68
...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86


In [6]:
# Data Preprocessing
# Merge transactions with product data to get product category information
transactions = transactions.merge(products[['ProductID', 'Category']], on='ProductID', how='left')

In [7]:
transactions

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Electronics
...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Electronics
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Electronics
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Electronics
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Electronics


In [8]:
# Create customer profile features
customer_profiles = transactions.groupby('CustomerID').agg(
    total_spend=('TotalValue', 'sum'),
    purchase_count=('TransactionID', 'count'),
    avg_purchase_value=('TotalValue', 'mean'),
    diversity_of_categories=('Category', lambda x: len(x.unique())),
    most_frequent_category=('Category', lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'),
    latest_transaction=('TransactionDate', lambda x: max(pd.to_datetime(x)))
).reset_index()


In [9]:
customer_profiles

,CustomerID,total_spend,purchase_count,avg_purchase_value,diversity_of_categories,most_frequent_category,latest_transaction
0,C0001,3354.52,5,670.904000,3,Electronics,2024-11-02 17:04:16
1,C0002,1862.74,4,465.685000,2,Clothing,2024-12-03 01:41:41
2,C0003,2725.38,4,681.345000,3,Home Decor,2024-08-24 18:54:04
3,C0004,5354.88,8,669.360000,3,Books,2024-12-23 14:13:52
4,C0005,2034.24,3,678.080000,2,Electronics,2024-11-04 00:30:22
...,...,...,...,...,...,...,...
194,C0196,4982.88,4,1245.720000,3,Home Decor,2024-12-15 03:43:35
195,C0197,1928.65,3,642.883333,2,Electronics,2024-12-27 18:20:31
196,C0198,931.83,2,465.915000,2,Clothing,2024-10-04 18:31:12
197,C0199,1979.28,4,494.820000,2,Electronics,2024-10-26 00:01:58


In [10]:
# Add recency feature (days since last transaction)
customer_profiles['recency_days'] = (pd.to_datetime('today') - customer_profiles['latest_transaction']).dt.days


In [11]:
# Merge with customer demographics
customer_profiles = customer_profiles.merge(customers[['CustomerID', 'Region', 'SignupDate']], on='CustomerID', how='left')

In [12]:
customer_profiles

,CustomerID,total_spend,purchase_count,avg_purchase_value,diversity_of_categories,most_frequent_category,latest_transaction,recency_days,Region,SignupDate
0,C0001,3354.52,5,670.904000,3,Electronics,2024-11-02 17:04:16,86,South America,2022-07-10
1,C0002,1862.74,4,465.685000,2,Clothing,2024-12-03 01:41:41,56,Asia,2022-02-13
2,C0003,2725.38,4,681.345000,3,Home Decor,2024-08-24 18:54:04,156,South America,2024-03-07
3,C0004,5354.88,8,669.360000,3,Books,2024-12-23 14:13:52,36,South America,2022-10-09
4,C0005,2034.24,3,678.080000,2,Electronics,2024-11-04 00:30:22,85,Asia,2022-08-15
...,...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,4,1245.720000,3,Home Decor,2024-12-15 03:43:35,44,Europe,2022-06-07
195,C0197,1928.65,3,642.883333,2,Electronics,2024-12-27 18:20:31,31,Europe,2023-03-21
196,C0198,931.83,2,465.915000,2,Clothing,2024-10-04 18:31:12,115,Europe,2022-02-27
197,C0199,1979.28,4,494.820000,2,Electronics,2024-10-26 00:01:58,94,Europe,2022-12-03


In [13]:
# Calculate customer age in days
customer_profiles['SignupDate'] = pd.to_datetime(customer_profiles['SignupDate'])
customer_profiles['customer_age'] = (pd.to_datetime('today') - customer_profiles['SignupDate']).dt.days


In [14]:
customer_profiles

,CustomerID,total_spend,purchase_count,avg_purchase_value,diversity_of_categories,most_frequent_category,latest_transaction,recency_days,Region,SignupDate,customer_age
0,C0001,3354.52,5,670.904000,3,Electronics,2024-11-02 17:04:16,86,South America,2022-07-10,933
1,C0002,1862.74,4,465.685000,2,Clothing,2024-12-03 01:41:41,56,Asia,2022-02-13,1080
2,C0003,2725.38,4,681.345000,3,Home Decor,2024-08-24 18:54:04,156,South America,2024-03-07,327
3,C0004,5354.88,8,669.360000,3,Books,2024-12-23 14:13:52,36,South America,2022-10-09,842
4,C0005,2034.24,3,678.080000,2,Electronics,2024-11-04 00:30:22,85,Asia,2022-08-15,897
...,...,...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,4,1245.720000,3,Home Decor,2024-12-15 03:43:35,44,Europe,2022-06-07,966
195,C0197,1928.65,3,642.883333,2,Electronics,2024-12-27 18:20:31,31,Europe,2023-03-21,679
196,C0198,931.83,2,465.915000,2,Clothing,2024-10-04 18:31:12,115,Europe,2022-02-27,1066
197,C0199,1979.28,4,494.820000,2,Electronics,2024-10-26 00:01:58,94,Europe,2022-12-03,787


In [15]:
customer_profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CustomerID               199 non-null    object        
 1   total_spend              199 non-null    float64       
 2   purchase_count           199 non-null    int64         
 3   avg_purchase_value       199 non-null    float64       
 4   diversity_of_categories  199 non-null    int64         
 5   most_frequent_category   199 non-null    object        
 6   latest_transaction       199 non-null    datetime64[ns]
 7   recency_days             199 non-null    int64         
 8   Region                   199 non-null    object        
 9   SignupDate               199 non-null    datetime64[ns]
 10  customer_age             199 non-null    int64         
dtypes: datetime64[ns](2), float64(2), int64(4), object(3)
memory usage: 17.2+ KB


In [16]:
# Drop unnecessary columns
customer_profiles.drop(columns=['SignupDate', 'latest_transaction'], inplace=True)


In [17]:
customer_profiles

,CustomerID,total_spend,purchase_count,avg_purchase_value,diversity_of_categories,most_frequent_category,recency_days,Region,customer_age
0,C0001,3354.52,5,670.904000,3,Electronics,86,South America,933
1,C0002,1862.74,4,465.685000,2,Clothing,56,Asia,1080
2,C0003,2725.38,4,681.345000,3,Home Decor,156,South America,327
3,C0004,5354.88,8,669.360000,3,Books,36,South America,842
4,C0005,2034.24,3,678.080000,2,Electronics,85,Asia,897
...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,4,1245.720000,3,Home Decor,44,Europe,966
195,C0197,1928.65,3,642.883333,2,Electronics,31,Europe,679
196,C0198,931.83,2,465.915000,2,Clothing,115,Europe,1066
197,C0199,1979.28,4,494.820000,2,Electronics,94,Europe,787


In [18]:
# Ensure categorical columns are strings
customer_profiles['Region'] = customer_profiles['Region'].astype(str)
customer_profiles['most_frequent_category'] = customer_profiles['most_frequent_category'].astype(str)


In [19]:
# Define Preprocessor for Categorical and Numeric Data
encoder = ColumnTransformer(
    transformers=[
        ('region', OneHotEncoder(sparse_output=False), ['Region']),
        ('most_frequent_category', OneHotEncoder(sparse_output=False), ['most_frequent_category'])
    ],
    remainder='passthrough'
)

# Continuous features to scale
scaler = StandardScaler()
continuous_columns = ['total_spend', 'purchase_count', 'avg_purchase_value', 
                      'diversity_of_categories', 'recency_days', 'customer_age']


In [20]:
# Transform data
transformed_data = encoder.fit_transform(customer_profiles.drop(columns=['CustomerID']))
encoded_columns = encoder.get_feature_names_out()
scaled_continuous_data = scaler.fit_transform(customer_profiles[continuous_columns])


In [21]:
# Combine transformed data
final_data = np.hstack([transformed_data, scaled_continuous_data])


In [22]:
final_data

array([[ 0.        ,  0.        ,  0.        , ...,  0.16054032,
        -0.26918431,  1.148752  ],
       [ 1.        ,  0.        ,  0.        , ..., -0.90437716,
        -0.69320286,  1.60043068],
       [ 0.        ,  0.        ,  0.        , ...,  0.16054032,
         0.72019231, -0.71327032],
       ...,
       [ 0.        ,  1.        ,  0.        , ..., -0.90437716,
         0.14070029,  1.55741366],
       [ 0.        ,  1.        ,  0.        , ..., -0.90437716,
        -0.15611269,  0.70014596],
       [ 1.        ,  0.        ,  0.        , ...,  1.22545781,
        -0.80627447,  0.11634358]])

In [23]:
# Similarity Calculation using Cosine Similarity
cosine_sim = cosine_similarity(final_data)

In [24]:
cosine_sim

array([[1.        , 0.9689315 , 0.98717627, ..., 0.83786846, 0.99362549,
        0.98957056],
       [0.9689315 , 1.        , 0.92212149, ..., 0.94414529, 0.98947762,
        0.92384755],
       [0.98717627, 0.92212149, 1.        , ..., 0.76124844, 0.96834854,
        0.99783891],
       ...,
       [0.83786846, 0.94414529, 0.76124844, ..., 1.        , 0.89380552,
        0.75507276],
       [0.99362549, 0.98947762, 0.96834854, ..., 0.89380552, 1.        ,
        0.968386  ],
       [0.98957056, 0.92384755, 0.99783891, ..., 0.75507276, 0.968386  ,
        1.        ]])

In [25]:
# Get top 3 similar customers for each customer (for C0001 - C0020)
top_3_lookalikes = {}

for i in range(20):  # First 20 customers (C0001 to C0020)
    customer_id = customer_profiles.loc[i, 'CustomerID']
    similarity_scores = cosine_sim[i]
    
    # Get top 3 most similar customers (excluding self)
    similar_customers_indices = np.argsort(similarity_scores)[::-1][1:4]
    similar_customers = [
        (customer_profiles.loc[j, 'CustomerID'], round(similarity_scores[j], 4))
        for j in similar_customers_indices
    ]
    top_3_lookalikes[customer_id] = similar_customers

In [26]:

# Save the map to Lookalike.csv
lookalike_map = []

for customer_id, similar_list in top_3_lookalikes.items():
    lookalike_map.append({
        'cust_id': customer_id,
        'lookalikes': similar_list
    })

lookalike_df = pd.DataFrame(lookalike_map)


In [27]:
lookalike_df

,cust_id,lookalikes
0,C0001,"[(C0174, 1.0), (C0106, 0.9999), (C0088, 0.9999)]"
1,C0002,"[(C0029, 0.9999), (C0025, 0.998), (C0121, 0.99..."
2,C0003,"[(C0052, 0.9999), (C0177, 0.9995), (C0031, 0.9..."
3,C0004,"[(C0104, 1.0), (C0165, 0.9999), (C0188, 0.9998)]"
4,C0005,"[(C0159, 0.9998), (C0176, 0.9992), (C0132, 0.9..."
5,C0006,"[(C0177, 1.0), (C0026, 0.9999), (C0092, 0.9999)]"
6,C0007,"[(C0132, 0.9994), (C0197, 0.9991), (C0074, 0.9..."
7,C0008,"[(C0156, 0.9998), (C0075, 0.9997), (C0141, 0.9..."
8,C0009,"[(C0077, 0.9996), (C0043, 0.9995), (C0071, 0.9..."
9,C0010,"[(C0166, 1.0), (C0094, 0.9995), (C0199, 0.999)]"


In [38]:
# Save the output to Lookalike.csv
lookalike_df.to_csv('Lookalike.csv', index=False)

print("Lookalike Model has been generated and saved to Lookalike.csv")

Lookalike Model has been generated and saved to Lookalike.csv
